In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import copy

In [ ]:
# -------------------------------------------------
# Constant
SIZE_FILTER_X = 36
SIZE_FILTER_Y = 36
NB_ITERATION = 5

In [ ]:
def search_visage(gray_image, size_filter_X, size_filter_Y):
    
    # Raise an exception, if we can't apply the fitlter
    width, height = gray_image.shape[0], gray_image.shape[1]
    
    if width < size_filter_Y and height < size_filter_X :
        raise Exception ("impossible to crop properly")
        
    if (size_filter_X/2) % 2 != 0 or (size_filter_Y/2) % 2 != 0:
        raise Exception ("All dimension of the filter should be pair")
    
    # loop on the image.
    fil_divi_2_X = int (size_filter_X/2)
    fil_divi_2_Y = int (size_filter_Y/2)
    result = []
    for y in range(fil_divi_2_X, height - fil_divi_2_X):
        for x in range(fil_divi_2_Y, width - fil_divi_2_Y):
            crop_img = gray_image[x - fil_divi_2_X: x + fil_divi_2_X, y - fil_divi_2_Y : y + fil_divi_2_Y]
            # apply the model
            
            # test --> rectangle.
            if x == 110 and y == 30: 
                result.append((x,y))  
    return result

In [ ]:
def draw_rectangle(img, cor_x, cor_y, size_filter_X, size_filter_Y ):
    fil_divi_2_X = int (size_filter_X/2)
    fil_divi_2_Y = int (size_filter_Y/2)
    cv2.rectangle(img, (cor_x - fil_divi_2_X, cor_y - fil_divi_2_Y), (cor_x + fil_divi_2_X, cor_y + fil_divi_2_Y), (0,255,255), 2)
    return img

In [ ]:
def pyramid_search (path_image, size_filter_X, size_filter_Y, nb_iterations):
    # We get and tranform the image in gray.
    img_search = cv2.imread(path_image)
    gray_image = cv2.cvtColor(img_search, cv2.COLOR_BGR2GRAY)
    gray_imag_rect  = copy.deepcopy(gray_image)
    
    # Make the reduction.
    # --> search
    height, width = gray_image.shape
    nbPixelToReduceX = int((width  - size_filter_X) / nb_iterations)
    nbPixelToReduceY = int((height - size_filter_Y) / nb_iterations)

    for ite in range(nb_iterations) :
        
        height, width = gray_image.shape
        
        if(ite == nb_iterations-1):
            gray_image = cv2.resize(gray_image, (size_filter_X, size_filter_Y)) # Last iteration
            size_filter_X_new = width - nbPixelToReduceX
            size_filter_Y_new = height - nbPixelToReduceY
        elif(ite == 0):
            size_filter_X_new = size_filter_X - nbPixelToReduceX
            size_filter_Y_new = size_filter_Y - nbPixelToReduceY
        else :
            gray_image = cv2.resize(gray_image, (width - nbPixelToReduceX, height - nbPixelToReduceY))   
        
        # we get all square that seems to be a visage.
        result = search_visage(gray_image, size_filter_X, size_filter_Y)
        
        size_filter_X_new = nbPixelToReduceX + size_filter_X_new
        size_filter_Y_new = nbPixelToReduceY + size_filter_Y_new
        
        
        # draw red rectangle.
        for x, y in result: 
            gray_imag_rect = draw_rectangle(gray_imag_rect, x, y, size_filter_X_new, size_filter_Y_new)
        
    return gray_imag_rect

In [ ]:
result = pyramid_search('hulk.jpg', SIZE_FILTER_X, SIZE_FILTER_Y, NB_ITERATION)

In [ ]:
plt.imshow(result)